### Download the [Piano Concerto Dataset (PCD)](link:https://audiolabs-erlangen.de/resources/MIR/PCD) and checkpoints

In [ ]:
# Download the test dataset PCD and unzip it
!wget https://www.audiolabs-erlangen.de/resources/MIR/PCD/PCD_1.0.0.zip
!unzip PCD_1.0.0.zip
!chmod -R 775 PCD_1.0.0
!rm PCD_1.0.0.zip

# Download the pre-trained weights
!gdown --folder 'https://drive.google.com/drive/folders/1-zcdkHWUcfehaTjoxp-eCjAjZevDGxSu' -O checkpoints

### Import packages, set constants

In [ ]:
import numpy as np
import os
from pathlib import Path
import torch
import torchaudio

from utils import audio_player_list, init_separator


TEST_AUDIO_DIR = 'PCD_1.0.0/excerpts/'

# MODEL_TYPE is to be chosen among UMX06, UMX20, SPL, DMC, HDMC
# For more information, see https://audiolabs-erlangen.de/resources/MIR/2023-PianoConcertoSeparation
MODEL_TYPE = 'HDMC'
DEVICE = 'cpu'
separator = init_separator(model_type=MODEL_TYPE,
                           device=DEVICE)

Fs = 44100

### Separate various piano concertos from [PCD](link:https://audiolabs-erlangen.de/resources/MIR/PCD)

In [ ]:
for concerto_id in sorted(os.listdir(TEST_AUDIO_DIR))[0::10]:
    audio, sr = torchaudio.load(os.path.join(TEST_AUDIO_DIR, 
                                             f'{concerto_id}',
                                             f'{concerto_id}_OP.wav'))

    estimates_dict = separator.separate(audio.unsqueeze(0))

    piano = estimates_dict['piano'].detach().numpy()
    orch = estimates_dict['orch'].detach().numpy()

    print(concerto_id)
    audio_player_list([audio.to('cpu').detach().numpy(),
                       piano,
                       orch], [Fs, Fs, Fs], width=180, height=30, 
                  columns=['mix', 'piano', 'orch'])

### Compare different models

In [ ]:
CONCERTO_ID = 'Schumann_Op054-01-mm019-024_ES-V1' # See also metadata.csv file

audio, sr = torchaudio.load(os.path.join(TEST_AUDIO_DIR, 
                                         f'{CONCERTO_ID}',
                                         f'{CONCERTO_ID}_OP.wav'))

for model in ['UMX06', 'UMX20', 'SPL', 'DMC', 'HDMC']:
    separator = init_separator(model_type=model,
                               device=DEVICE)
    
    estimates_dict = separator.separate(audio.unsqueeze(0))

    piano = estimates_dict['piano'].detach().numpy()
    orch = estimates_dict['orch'].detach().numpy()

    print(model)
    audio_player_list([audio.to('cpu').detach().numpy(),
                       piano,
                       orch], [Fs, Fs, Fs], width=180, height=30, 
                      columns=['mix', 'piano', 'orch'])    